In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=c72b0b7d7f8a3a365d39dedd231a1db96e3239d622e004751bf3a85da1bf74e4
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
import os
import shutil

os.makedirs('inputFiles')

for f in os.listdir():
  if f.startswith('tweets'):
    shutil.move('/content/'+f, '/content/inputFiles/'+f)

In [3]:
def get_hashtags(line):
  words = line.split(' ')

  hashtags = []

  for word in words:
    if word.startswith('#'):
      hashtags.append((word, 1))

  return hashtags

In [8]:
from pyspark.streaming import StreamingContext
from pyspark import *

ssc = StreamingContext(sc, 10)

tweetsDStream = ssc.textFileStream('inputFiles/')

hashtagsMappedByOne = tweetsDStream.flatMap(get_hashtags)
hashtagsMappedByKey = hashtagsMappedByOne.reduceByKeyAndWindow(lambda v1, v2: v1+v2, lambda vnow, vold: vnow-vold, 30, 10)
hashtagsSorted = hashtagsMappedByKey.filter(lambda pair: pair[1]>=100)\
                  .transform(lambda batchRDD: batchRDD.sortBy(lambda pair: -1*pair[1]))
hashtagsSorted.saveAsTextFiles("out")
hashtagsSorted.pprint(10)

ssc.checkpoint("checkpointfolder");
ssc.start()

Py4JJavaError: ignored

In [9]:
sc.stop()
ssc.stop(stopSparkContext=False)